In [ ]:
import os
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import implicit
import pickle
import scipy.sparse as sparse
from surprise.model_selection import GridSearchCV
from catboost import CatBoostRegressor
from flask import Flask, request, jsonify
from pyngrok import ngrok

#### Загрузка данных

In [ ]:
os.makedirs('data', exist_ok=True)
os.makedirs('models', exist_ok=True)

In [ ]:
# Загружаем данные
u1_base = pd.read_csv('data/u1.base', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
u1_test = pd.read_csv('data/u1.test', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

# Оставляем нужные колонки
u1_base = u1_base.drop(columns='timestamp')
u1_test = u1_test.drop(columns='timestamp')

print(f"Train set shape: {u1_base.shape}, Test set shape: {u1_test.shape}")

In [ ]:
column_names = ['item_id', 'title' , 'release date','video_release_date',
              'IMDb URL', 'unknown','Action', 'Adventure','Animation',
              'Children', 'Comedy' , 'Crime','Documentary', 'Drama', 'Fantasy',
              'Film-Noir', 'Horror' , 'Musical', 'Mystery', 'Romance','Sci-Fi',
              'Thriller', 'War', 'Western']

# Загружаем файл, разделитель — табуляция ('\t')
item_df = pd.read_csv('data/u.item', sep='|', names=column_names, encoding='latin-1')
movies = item_df[['item_id', 'title']]


In [ ]:
users = pd.read_csv('data/u.user', sep='|', names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])

#### Коллаборативная фильтрация с использованием Surprise (SVD)

- *Surprise* — это Python-библиотека, созданная специально для построения и оценки рекомендательных систем. Она предоставляет множество алгоритмов для коллаборативной фильтрации, включая SVD (Singular Value Decomposition), один из самых популярных методов матричной факторизации.

- *SVD* — метод матричной факторизации, который разлагает матрицу пользователь-товар на две низкоразмерные матрицы (пользователи и товары), позволяя прогнозировать скрытые предпочтения и рекомендовать новые объекты на основе этих факторов. Обучается он итеративно с оптимизацией ошибки предсказания по известным рейтингам пользователей.

In [ ]:
reader = Reader(rating_scale=(1, 5)) # Создаем объект Reader, который задает шкалу рейтингов от 1 до 5

dataset_train = Dataset.load_from_df(u1_base[['user_id', 'item_id', 'rating']], reader)
trainset = dataset_train.build_full_trainset()# Строим объект trainset для обучения

# Создаем тестовый датасет в виде списка кортежей (user_id, item_id, рейтинг) из u1_test
testset = list(zip(u1_test['user_id'], u1_test['item_id'], u1_test['rating']))

svd = SVD(n_factors=50, reg_all=0.02, random_state=42)
svd.fit(trainset)

svd_predictions = svd.test(testset)# predict только для одной пары,а test() для всего набора; возвращает список объектов(uid-user_id,iid-item_id,r_ui-реальный рейтинг,est-предсказанный рейтинг)
y_true = [pred.r_ui for pred in svd_predictions]
y_pred = [pred.est for pred in svd_predictions]

rmse_svd = np.sqrt(mean_squared_error(y_true, y_pred))
mae_svd = mean_absolute_error(y_true, y_pred)

print(f'SVD RMSE: {rmse_svd:.4f}, MAE: {mae_svd:.4f}')

# Сохраняем модель
with open('svd_model.pkl', 'wb') as f_out:
    pickle.dump(svd, f_out)


#### Коллаборативная фильтрация с implicit (ALS)

- Библиотека Implicit применяет модели коллаборативной фильтрации, оптимизированные для работы с неявными данными (просмотры, клики, покупки без рейтингов). В основе часто лежит матричная факторизация ALS (Alternating Least Squares), которая эффективно работает с большими разреженными данными и минимизирует ошибку для неявных сигналов взаимодействия.

In [ ]:
user_item = sparse.coo_matrix((np.ones(len(u1_base)), (u1_base['user_id'], u1_base['item_id'])))
user_item_csr = user_item.tocsr()

# Инициализация модели ALS
model_als = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.01, iterations=20)

# Обучение модели
model_als.fit(user_item_csr.T)

# Функция предсказания (без .get() — обычные numpy массивы)
def predict_als(user, item):
    user_vec = model_als.user_factors[user]
    item_vec = model_als.item_factors[item]
    return np.dot(user_vec, item_vec)

# Оценка модели
als_preds = []
for idx, row in u1_test.iterrows():
    user = row['user_id']
    item = row['item_id']
    true_rating = row['rating']
    try:
        pred_rating = predict_als(user, item)
    except IndexError:
        pred_rating = np.mean(u1_base['rating'])
    als_preds.append((true_rating, pred_rating))

y_true_als = [x[0] for x in als_preds]
y_pred_als = [x[1] for x in als_preds]

from sklearn.metrics import mean_squared_error, mean_absolute_error
rmse_als = np.sqrt(mean_squared_error(y_true_als, y_pred_als))
mae_als = mean_absolute_error(y_true_als, y_pred_als)

print(f'Implicit ALS GPU RMSE: {rmse_als:.4f}, MAE: {mae_als:.4f}')

####  Ансамблирование и улучшение моделей

##### Подбор гиперпараметров для SVD с GridSearchCV (Surprise)

In [ ]:
reader = Reader(rating_scale=(1, 5))
data_surprise = Dataset.load_from_df(u1_base[['user_id', 'item_id', 'rating']], reader)

param_grid = {
    'n_factors': [20, 50, 100],
    'reg_all': [0.02, 0.05],
    'lr_all': [0.002, 0.005]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data_surprise)

print("Лучшие параметры SVD:", gs.best_params['rmse'])
print("Лучшее RMSE (CV):", gs.best_score['rmse'])

# Обучаем лучшую модель SVD
best_svd = gs.best_estimator['rmse']
trainset = data_surprise.build_full_trainset()
best_svd.fit(trainset)

# Предсказания SVD на тесте
testset = list(zip(u1_test['user_id'], u1_test['item_id'], u1_test['rating']))
svd_predictions = best_svd.test(testset)
y_true_svd = [pred.r_ui for pred in svd_predictions]
y_pred_svd = [pred.est for pred in svd_predictions]

rmse_svd = np.sqrt(mean_squared_error(y_true_svd, y_pred_svd))
mae_svd = mean_absolute_error(y_true_svd, y_pred_svd)

print(f"SVD RMSE: {rmse_svd:.4f}, MAE: {mae_svd:.4f}")

##### Ансамблирование предсказаний (усреднение SVD и implicit)

In [ ]:
#  best_svd уже обучена с использованием GridSearchCV
# - model_als —  обученная модель implicit ALS

# 1. Получаем предсказания из SVD
testset = list(zip(u1_test['user_id'], u1_test['item_id'], u1_test['rating']))
svd_preds = best_svd.test(testset)
y_true = [pred.r_ui for pred in svd_preds]
y_pred_svd = [pred.est for pred in svd_preds]

# 2. Получаем предсказания из ALS
def predict_als(user, item):
    try:
        return model_als.user_factors[user].dot(model_als.item_factors[item])
    except IndexError:
        return np.mean(u1_test['rating'])  # средний рейтинг если отсутствуют

y_pred_als = [predict_als(row['user_id'], row['item_id']) for _, row in u1_test.iterrows()]

# 3. Усреднение предсказаний
ensemble_pred = [(svd + als) / 2 for svd, als in zip(y_pred_svd, y_pred_als)]

# 4. Оценка RMSE и MAE
rmse_ens = np.sqrt(mean_squared_error(y_true, ensemble_pred))
mae_ens = mean_absolute_error(y_true, ensemble_pred)
print(f"Ансамбль RMSE: {rmse_ens:.4f}, MAE: {mae_ens:.4f}")

##### Добавление CatBoost на признаки пользователей

- *CatBoost* — градиентный бустинг на деревьях решений, который может использоваться для рекомендательных систем, строя модель, предсказывающую рейтинг или вероятность взаимодействия, учитывая признаки пользователей, товаров и контекста. Подходит для гибридных рекомендаций с использованием табличных данных и категориальных признаков.

In [ ]:
train_df = u1_base.merge(users, on='user_id')
test_df = u1_test.merge(users, on='user_id')

features_to_drop = ['rating', 'user_id', 'item_id']

X_train = train_df.drop(columns=features_to_drop)
y_train = train_df['rating']

X_test = test_df.drop(columns=features_to_drop)
y_test = test_df['rating']

# Определяем категориальные признаки автоматически
cat_features = X_train.select_dtypes(include=['object']).columns.tolist()
cat_features_indices = [X_train.columns.get_loc(col) for col in cat_features]

model_cb = CatBoostRegressor(iterations=300, cat_features=cat_features_indices, verbose=100, random_seed=42)
model_cb.fit(X_train, y_train)

cb_preds = model_cb.predict(X_test)

rmse_cb = np.sqrt(mean_squared_error(y_test, cb_preds))
mae_cb = mean_absolute_error(y_test, cb_preds)

print(f'CatBoost RMSE: {rmse_cb:.4f}, MAE: {mae_cb:.4f}')

In [ ]:
data = {
    'Метод': ['Surprise(SVD)', 'Implicit ALS', 'GridSearchCV (Surprise)','Ансамбль','CatBoost'],
    'RMSE': [0.9519, 3.5127, 0.9519,1.9413,1.0648],
    'MAE': [0.7510, 3.2531,0.7521,1.7189,0.8541]
}

df = pd.DataFrame(data)
df.style.highlight_min(subset=['RMSE', 'MAE'], color='lightgreen')

Вывод:
- лучше всего справилась библиотека Surprise, на втором месте CatBoost.
- Implicit ALS показал низкий результат и ансамбль с ним только ухудшает значение метрики.
- для библиотеки Surprise уже изначально были использованы лучшие гиперпараметры, поэтому GridSearchCV не улучшил модель.

#### Сервис на Flask

In [ ]:
# Читаем исходные данные u1_base (должен быть DataFrame с user_id, item_id, rating)
reader = Reader(rating_scale=(1, 5))
dataset = Dataset.load_from_df(u1_base[['user_id', 'item_id', 'rating']], reader)

# Строим trainset для работы с моделью
trainset = dataset.build_full_trainset()

print("Trainset создан")

In [ ]:

# Вставьте сюда свой токен с https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = TOKEN
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

app = Flask(__name__)

# Загрузка модели
with open('/content/drive/MyDrive/svd_model.pkl', 'rb') as f_in:
    model = pickle.load(f_in)

# Загрузка таблицы с фильмами
movies = pd.read_csv('/content/drive/MyDrive/movies.csv')

def get_top_n_recommendations(algo, user_id, trainset, n=10):
    all_items = set(trainset.all_items())
    try:
        inner_uid = trainset.to_inner_uid(user_id)
        seen = {j for (j, _) in trainset.ur[inner_uid]}
    except ValueError:
        seen = set()  # Пользователь не найден, считаем, что фильмы не были просмотрены
        # Извлекаем все сырые рейтинги из trainset
        ratings = [(trainset.to_raw_iid(iid), rating) for (uid, iid, rating) in trainset.all_ratings()]
        ratings_df = pd.DataFrame(ratings, columns=['item_id', 'rating'])
        # Получаем средний рейтинг на item_id
        ratings_agg = ratings_df.groupby('item_id', as_index=False)['rating'].mean()
        merged = movies.merge(ratings_agg, on='item_id', how='left')
        popular = merged.sort_values(by='rating', ascending=False).head(n)
        return popular[['title', 'rating']].to_dict(orient='records')

    predictions = []
    for item_id in all_items - seen:
        pred = algo.predict(user_id, trainset.to_raw_iid(item_id))
        predictions.append((trainset.to_raw_iid(item_id), pred.est))
    predictions.sort(key=lambda x: x[1], reverse=True)

    top_df = pd.DataFrame(predictions[:n], columns=['item_id', 'predicted_rating'])
    top_df = top_df.merge(movies, on='item_id', how='left')
    return top_df[['title', 'predicted_rating']].to_dict(orient='records')

@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = request.args.get('user_id', type=int)
    recommendations = get_top_n_recommendations(model, user_id, trainset, n=10)
    return jsonify(recommendations)

if __name__ == '__main__':
    port = 5000
    public_url = ngrok.connect(port).public_url
    print(f"Ngrok tunnel URL: {public_url}")
    app.run(port=port)